In [ ]:
import os
import pickle

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from tensorflow import keras
from tensorflow.keras import layers

import sys
sys.path.append('../')
from scripts.get_s2_data_ee import get_history, get_history_polygon, get_pixel_vectors

%load_ext autoreload
%autoreload 2

In [ ]:
# Sentinel 2 band descriptions
band_descriptions = {
    'B1': 'Aerosols, 442nm',
    'B2': 'Blue, 492nm',
    'B3': 'Green, 559nm',
    'B4': 'Red, 665nm',
    'B5': 'Red Edge 1, 704nm',
    'B6': 'Red Edge 2, 739nm',
    'B7': 'Red Edge 3, 779nm',
    'B8': 'NIR, 833nm',
    'B8A': 'Red Edge 4, 864nm',
    'B9': 'Water Vapor, 943nm',
    'B11': 'SWIR 1, 1610nm',
    'B12': 'SWIR 2, 2186nm'
}

band_wavelengths = [442, 492, 559, 665, 704, 739, 779, 833, 864, 943, 1610, 2186]

In [ ]:
data_dir = '../data/training_sites'

with open(os.path.join(data_dir, "positive_data_toa.pkl"), 'rb') as file:
    positive = pickle.load(file)
    
with open(os.path.join(data_dir, "negative_data_toa.pkl"), 'rb') as file:
    negative = pickle.load(file)
    
data = np.concatenate((positive, negative))

In [ ]:
band_names = list(band_descriptions.keys())

In [ ]:
pd.DataFrame(data, columns=band_names).corr().style.background_gradient(cmap='coolwarm').set_precision(2)

In [ ]:
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import umap
reduced = PCA(n_components=1).fit_transform(data[:, 1:4])

In [ ]:
pd.DataFrame(np.array([data[:,7], reduced[:,0]]).T, columns=['B8', 'RGB']).corr()

In [ ]:
plt.figure(figsize=(8,8), facecolor=(1,1,1))
plt.scatter(data[:,7], data[:,8], s=0.1, alpha=0.5)
plt.plot([0, 1], [0, 1], c='r', alpha=0.4)
#plt.xlim((0,0.8))
#plt.ylim((0,0.8))
plt.ylabel('RGB PCA')
plt.xlabel('IR')
plt.title('Actual NDVI vs. Predicted NDVI')
plt.show()

In [ ]:
source = data[:,1:4] / 5000
target = (data[:,7] - data[:,3]) / (data[:,7] + data[:,3])
x, y = shuffle(source, target, random_state=42)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=42)

In [ ]:
input_shape = np.shape(x_train[0])
model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Dense(64, activation="relu"),
        layers.Dense(64, activation="relu"),
        layers.Dense(64, activation="relu"),
        layers.Dense(64, activation="relu"),
        layers.Dense(64, activation="relu"),
        layers.Dense(1, activation="linear"),
    ]
)
model.summary()

In [ ]:
model.compile(loss="mse", 
              optimizer="adam", 
              metrics=["mse", keras.metrics.RootMeanSquaredError()]
             )

In [ ]:
batch_size = 128
epochs = 50
model.fit(x_train, 
          y_train, 
          batch_size=batch_size, 
          epochs=epochs, 
          validation_data = (x_test, y_test)
         )

In [ ]:
# Enter rect width in degrees (0.035 max recommended) and site coordinates
rect_width = 0.025
site_coords = [115.350242, -8.562121]
name = 'temesi'
start_date = '2019-01-01'
patch_history = get_history([site_coords], 
                            [name], 
                            rect_width,
                            num_months = 12,
                            start_date = start_date)

In [ ]:
site_data = patch_history['2019-06-01']['temesi']
rgb = np.stack((site_data['B4'], site_data['B3'], site_data['B2']), axis=-1)
ndvi = (site_data['B8'] - site_data['B4']) / (site_data['B8'] + site_data['B4'])

In [ ]:
test = np.stack((site_data['B2'].flatten(), site_data['B3'].flatten(), site_data['B4'].flatten()), axis=-1)

In [ ]:
width, height = site_data['B4'].shape

In [ ]:
(pred_ndvi).min()

In [ ]:
plt.figure(figsize=(8,8), facecolor=(1,1,1))
plt.scatter(ndvi, pred_ndvi, s=0.1, alpha=0.5)
plt.plot([0, 1], [0, 1], c='r', alpha=0.4)
plt.xlim((0,0.8))
plt.ylim((0,0.8))
plt.ylabel('Predicted NDVI')
plt.xlabel('Actual NDVI')
plt.title('Actual NDVI vs. Predicted NDVI')
plt.show()

In [ ]:
plt.figure(figsize=(8,8), dpi=150)
plt.subplot(2,2,1)
plt.imshow((rgb / 3000), vmin=0, vmax=1)
plt.title('RGB')
plt.axis('off')
plt.subplot(2,2,2)
plt.imshow(ndvi, cmap='RdYlGn', vmin=-0.25, vmax=1)
plt.title('NDVI')
plt.colorbar()
plt.axis('off')
plt.subplot(2,2,3)
pred_ndvi = model.predict(test / 5000)[:,0].reshape(width, width)
plt.imshow(pred_ndvi - ndvi, cmap='RdYlGn', vmin=-1, vmax=1)
plt.title('Delta NDVI')
plt.colorbar()
plt.axis('off')
plt.subplot(2,2,4)
pred_ndvi = model.predict(test / 5000)[:,0].reshape(width, width)
plt.imshow(pred_ndvi, cmap='RdYlGn', vmin=-0.25, vmax=1)
plt.title('Pred NDVI')
plt.colorbar()
plt.axis('off')
plt.show()

In [ ]:
plt.figure(figsize=(12,4), dpi=150, facecolor=(1,1,1))
plt.subplot(1,3,1)
plt.imshow(ndvi, cmap='RdYlGn', vmin=-0.25, vmax=1)
plt.title('Actual NDVI')
plt.colorbar()
plt.axis('off')

plt.subplot(1,3,2)
pred_ndvi = model.predict(test / 5000)[:,0].reshape(width, width)
plt.imshow(pred_ndvi, cmap='RdYlGn', vmin=-0.25, vmax=1)
plt.title('Pred NDVI')
plt.colorbar()
plt.axis('off')
plt.subplot(1,3,3)

pred_ndvi = model.predict(test / 5000)[:,0].reshape(width, width)
plt.imshow(pred_ndvi - ndvi, cmap='RdYlGn', vmin=-1, vmax=1)
plt.title('Delta NDVI')
plt.colorbar()
plt.axis('off')

plt.show()

In [ ]:
plt.imshow(ndvi - pred_ndvi)
plt.colorbar()
plt.show()